In [5]:
# Imports
import open3d as o3d
import numpy as np
import torch
import csv
import os

# To convert to a .py file: jupyter nbconvert --to python model_evaluation.ipynb
DEBUG=True

pcd_file_path='/home/hoch/dev/pcd2bin/pcd/garage_color.pcd'
csv_file_path='/home/hoch/dev/pcd2bin/pcd/meta.csv'
bin_file_path='/home/hoch/dev/garage.bin'
pcd_file = o3d.io.read_point_cloud(pcd_file_path)

# pcd_file_path='/home/hoch/dev/pcd2bin/pcd/garage.pcd'
# pcd_file=pypcd.PointCloud.from_path(pcd_file_path)
# print(pcd_file_path)
# print(pcd_file.__dict__)

# pcd_file_path='/home/hoch/dev/pcd2bin/pcd/garage_color.pcd'
# pcd_file=pypcd.PointCloud.from_path(pcd_file_path)
# print(pcd_file_path)
# print(pcd_file.__dict__)

In [ ]:
if(DEBUG):
    print(np.asarray(pcd_file.points).shape)
    o3d.visualization.draw_geometries([pcd_file])

    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = pcd_file.voxel_down_sample(voxel_size=0.05)
    o3d.visualization.draw_geometries([downpcd])

    print("Recompute the normal of the downsampled point cloud")
    downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.1, max_nn=30))
    o3d.visualization.draw_geometries([downpcd])

In [7]:
new_pcd = o3d.t.io.read_point_cloud(pcd_file_path)
positions  = new_pcd.point['positions'].numpy().astype(np.float32)
colors     = new_pcd.point['colors'].numpy().astype(np.float32)
# normals = new_pcd.point['normals']


In [65]:
x_values = positions[:, 0]
y_values = positions[:, 1]
z_values = positions[:, 2]
# https://docs.agi32.com/AGi32/Content/multi_use_forms/Reflectance_and_Color_Selection_-_Concepts.htm
# Y = 0.2126 * R + 0.7152 * G + 0.0722 * B
l_values = 0.2126 * colors[:, 0] + \
           0.7152 * colors[:, 1] + \
           0.0722 * colors[:, 2]

# Scale [0, 255] range to [0, 1]
i_values = l_values / 255.0

new_points = np.transpose(np.vstack((x_values, y_values, z_values, i_values)))
new_points.tofile(bin_file_path)


(125826, 4)
(125635, 4)


In [61]:
# DEBUG

INPUT_IMAGES=['/home/hoch/dev/garage.bin', # Custom point cloud file
              'data/cmpe249-fa22/kitti/testing/velodyne/000000.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000001.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000002.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000003.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000004.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000005.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000006.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000007.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000008.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000009.bin']

for image in INPUT_IMAGES:
    current_file = np.fromfile(image, dtype='float32')
    print(f'image: {image} -- shape: {current_file.shape}')
    print(current_file[0:4])

# bad_file  = np.fromfile(INPUT_IMAGES[0], dtype='float32')
# good_file = np.fromfile(INPUT_IMAGES[1], dtype='float32')

image: data/cmpe249-fa22/kitti/testing/velodyne/000000.bin -- shape: (502540,)
[5.5025e+01 2.5000e-02 2.0700e+00 0.0000e+00]
image: /home/hoch/dev/garage.bin -- shape: (503304,)
[-46.654545  -26.063972  -62.99285     0.6508251]
image: data/cmpe249-fa22/kitti/testing/velodyne/000001.bin -- shape: (488436,)
[30.458  5.764  1.257  0.08 ]
image: data/cmpe249-fa22/kitti/testing/velodyne/000002.bin -- shape: (462416,)
[75.692  3.495  2.771  0.   ]
image: data/cmpe249-fa22/kitti/testing/velodyne/000003.bin -- shape: (487180,)
[47.187  8.56   1.831  0.   ]
image: data/cmpe249-fa22/kitti/testing/velodyne/000004.bin -- shape: (498256,)
[37.786  7.897  1.514  0.94 ]
image: data/cmpe249-fa22/kitti/testing/velodyne/000005.bin -- shape: (439092,)
[4.1405e+01 1.3000e-02 1.6090e+00 0.0000e+00]
image: data/cmpe249-fa22/kitti/testing/velodyne/000006.bin -- shape: (449772,)
[76.231 23.11   2.903  0.   ]
image: data/cmpe249-fa22/kitti/testing/velodyne/000007.bin -- shape: (460944,)
[26.729  0.083  1.113  